### Different Zone or Same Zone Test Observability

Steps to get observability
*   Get test duration (from propagation logs)
*   Get time series metric values (based on CPU usage, Bytes Transmitted, Bytes Received, and Memory usage)
*   Get metrix values that match time series   



Get all the required libraries

In [ ]:
import pandas as pd
import numpy as np
import json, re
from pandas import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import gaussian_kde

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Go to root directory. This directory will depend on the location of your dataset (results).

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/geographical

/content/drive/My Drive/Colab Notebooks/bcgossip/sim/gpbc/cnsim_plosone/geographical


Get bandwidth test duration by loading memory propagation time. This time duration will be used to extract metrics from metrics explorer.  

In [ ]:
def get_test_duration(file_path):
    """
    Calculates the duration of a test from a CSV file.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pandas.DataFrame or None: A DataFrame containing the number of nodes,
                                  minimum and maximum datetimes in MYT, or None
                                  if an error occurs.
    """
    # Get source file
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    try:
        # Remove rows where 'message' ends with "-0" (convergence phase)
        df = df[~df['message'].str.endswith("-0")]

        # Extract number of nodes from 'message'
        df['num_nodes'] = df['message'].str.extract(r'cubaan(\d+)-')[0].astype(int)

        # Group by 'num_nodes' and find the min and max of 'received_timestamp'
        duration_df = df.groupby('num_nodes')['received_timestamp'].agg(['min', 'max']).reset_index()

        # Convert min and max columns to datetime (assuming nanoseconds)
        duration_df['min_datetime_myt'] = pd.to_datetime(duration_df['min'], unit='ns')
        duration_df['max_datetime_myt'] = pd.to_datetime(duration_df['max'], unit='ns')

        # Convert to MYT
        duration_df['min_datetime_myt'] = duration_df['min_datetime_myt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')
        duration_df['max_datetime_myt'] = duration_df['max_datetime_myt'].dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')

        # Format the datetime objects as "M/d/YYYY HH:MM:SS"
        duration_df['min_datetime_myt'] = duration_df['min_datetime_myt'].dt.strftime('%-m/%-d/%Y %H:%M:%S')
        duration_df['max_datetime_myt'] = duration_df['max_datetime_myt'].dt.strftime('%-m/%-d/%Y %H:%M:%S')

        return duration_df

    except KeyError as e:
        print(f"Error: Column not found. {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
## Get Zonal test duration
df_zonal_duration = get_test_duration('test-zonal-10X.csv')
df_zonal_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46
1,50,1.742906e+18,1.742906e+18,3/25/2025 20:37:00,3/25/2025 20:38:21
2,100,1.742907e+18,1.742907e+18,3/25/2025 20:49:03,3/25/2025 20:50:26
3,200,1.742908e+18,1.742908e+18,3/25/2025 21:06:21,3/25/2025 21:07:55
4,400,1.742908e+18,1.742908e+18,3/25/2025 21:10:47,3/25/2025 21:12:45
5,600,1.742909e+18,1.742909e+18,3/25/2025 21:16:39,3/25/2025 21:18:59


In [ ]:
## Get Regional test duration
df_regional_duration = get_test_duration('test-regional-10X.csv')
df_regional_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt
0,10,1.742900e+18,1.742900e+18,3/25/2025 18:46:25,3/25/2025 18:47:37
1,50,1.742900e+18,1.742900e+18,3/25/2025 18:51:35,3/25/2025 18:52:55
2,100,1.742876e+18,1.742876e+18,3/25/2025 12:18:30,3/25/2025 12:19:54
3,200,1.742902e+18,1.742902e+18,3/25/2025 19:19:21,3/25/2025 19:20:52
4,400,1.742902e+18,1.742902e+18,3/25/2025 19:24:20,3/25/2025 19:26:22
5,600,1.742903e+18,1.742903e+18,3/25/2025 19:44:41,3/25/2025 19:47:01


In [ ]:
def get_timeseries(file_path):
    """
    Loads a CSV file containing time series data. Converts the 'TimeSeries ID'
    column to datetime objects in Asia/Kuala_Lumpur timezone, and returns the
    processed time series DataFrame.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pandas.DataFrame: The DataFrame with the 'TimeSeries ID' column converted to datetime.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

    # Define the datetime format string to match the data
    datetime_format = '%a %b %d %Y %H:%M:%S GMT%z (Malaysia Time)'

    try:
        # Convert 'TimeSeries ID' to datetime, convert to MYT, and format
        df['TimeSeries ID'] = (pd.to_datetime(df['TimeSeries ID'], format=datetime_format, utc=True)
                            .dt.tz_convert('Asia/Kuala_Lumpur')
                            .dt.strftime('%m/%d/%Y %H:%M:%S'))

        # Convert 'TimeSeries ID' to datetime in df
        df['TimeSeries ID'] = pd.to_datetime(df['TimeSeries ID'])
    except KeyError:
        print("Error: 'TimeSeries ID' column not found in the CSV file.")
        return None
    except ValueError as e:
        print(f"Error: Datetime conversion failed. {e}")
        return None

    return df

a. Observability for Bytes transmitted (with PromQL filtering)
```
sum(rate(kubernetes_io:pod_network_sent_bytes_count{monitored_resource="k8s_pod"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.



In [ ]:
# Load Bytes transmitted csv file (from metrix explorer) for zonal and regional tests
# Get these tests min datetime value and max datetime value and key in this in metrics explorer
# together with (a) PromQL command

df_zonal_transmit = get_timeseries('zonalBytestransmitted.csv')
# df_zonal_transmit

df_regional_transmit = get_timeseries('regionalBytestransmitted.csv')
# df_regional_transmit

In [ ]:
# Function to find the max 'telemetry-explorer-0-0' value within a given time range
def find_max_telemetry(min_time, max_time, df_util):
    filtered_df = df_util[(df_util['TimeSeries ID'] >= min_time) & (df_util['TimeSeries ID'] <= max_time)]
    if not filtered_df.empty:
        return filtered_df['telemetry-explorer-0-0'].max()
    else:
        return None  # Return None if no data within the range

In [ ]:
## Getting Bytes transmitted for Zonal Test

# Apply the function to each row in grouped_df
df_zonal_duration['bytes_transmit_zonal'] = df_zonal_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_zonal_transmit), axis=1
)

df_zonal_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_zonal
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,9148469248
1,50,1.742906e+18,1.742906e+18,3/25/2025 20:37:00,3/25/2025 20:38:21,11995721728
2,100,1.742907e+18,1.742907e+18,3/25/2025 20:49:03,3/25/2025 20:50:26,14524289024
3,200,1.742908e+18,1.742908e+18,3/25/2025 21:06:21,3/25/2025 21:07:55,20613545984
4,400,1.742908e+18,1.742908e+18,3/25/2025 21:10:47,3/25/2025 21:12:45,30477279232
5,600,1.742909e+18,1.742909e+18,3/25/2025 21:16:39,3/25/2025 21:18:59,42199904256


In [ ]:
## Getting Bytes transmitted for Regional Test

# Apply the function to each row in grouped_df
df_regional_duration['bytes_transmit_regional'] = df_regional_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_regional_transmit), axis=1
)

df_regional_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_regional
0,10,1.742900e+18,1.742900e+18,3/25/2025 18:46:25,3/25/2025 18:47:37,10142011392
1,50,1.742900e+18,1.742900e+18,3/25/2025 18:51:35,3/25/2025 18:52:55,11428057088
2,100,1.742876e+18,1.742876e+18,3/25/2025 12:18:30,3/25/2025 12:19:54,26949328896
3,200,1.742902e+18,1.742902e+18,3/25/2025 19:19:21,3/25/2025 19:20:52,20127506432
4,400,1.742902e+18,1.742902e+18,3/25/2025 19:24:20,3/25/2025 19:26:22,32191254528
5,600,1.742903e+18,1.742903e+18,3/25/2025 19:44:41,3/25/2025 19:47:01,43523231744


b. Observability for Bytes received (with PromQL filtering)
```
sum(rate(kubernetes_io:pod_network_received_bytes_count{monitored_resource="k8s_pod"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load Bytes received csv files (from metrix explorer) for default zonal and regional tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_zonal_received = get_timeseries('zonalBytesReceived.csv')
# df_zonal_received

df_regional_received = get_timeseries('regionalBytesReceived.csv')
# df_regional_received

In [ ]:
## Getting Bytes received for zonal test

# Apply the function to each row in grouped_df
df_zonal_duration['bytes_received_zonal'] = df_zonal_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_zonal_received), axis=1
)

# Remove a column inplace
# df_150Mi_duration.drop('bytes_received_default', axis=1, inplace=True)
# Select the desired columns using a list
df_zonal_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_zonal,bytes_received_zonal
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,9148469248,7.602844e+07
1,50,1.742906e+18,1.742906e+18,3/25/2025 20:37:00,3/25/2025 20:38:21,11995721728,2.795578e+08
2,100,1.742907e+18,1.742907e+18,3/25/2025 20:49:03,3/25/2025 20:50:26,14524289024,9.674019e+05
3,200,1.742908e+18,1.742908e+18,3/25/2025 21:06:21,3/25/2025 21:07:55,20613545984,1.331341e+07
4,400,1.742908e+18,1.742908e+18,3/25/2025 21:10:47,3/25/2025 21:12:45,30477279232,2.409727e+06
5,600,1.742909e+18,1.742909e+18,3/25/2025 21:16:39,3/25/2025 21:18:59,42199904256,3.105526e+06


In [ ]:
## Getting Bytes received for regional test

# Apply the function to each row in grouped_df
df_regional_duration['bytes_received_regional'] = df_regional_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_regional_received), axis=1
)

# Remove a column inplace
# df_150Mi_duration.drop('bytes_received_default', axis=1, inplace=True)
# Select the desired columns using a list
df_regional_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_regional,bytes_received_regional
0,10,1.742900e+18,1.742900e+18,3/25/2025 18:46:25,3/25/2025 18:47:37,10142011392,3.095369e+07
1,50,1.742900e+18,1.742900e+18,3/25/2025 18:51:35,3/25/2025 18:52:55,11428057088,1.192209e+08
2,100,1.742876e+18,1.742876e+18,3/25/2025 12:18:30,3/25/2025 12:19:54,26949328896,2.055274e+06
3,200,1.742902e+18,1.742902e+18,3/25/2025 19:19:21,3/25/2025 19:20:52,20127506432,1.207632e+07
4,400,1.742902e+18,1.742902e+18,3/25/2025 19:24:20,3/25/2025 19:26:22,32191254528,2.644369e+06
5,600,1.742903e+18,1.742903e+18,3/25/2025 19:44:41,3/25/2025 19:47:01,43523231744,3.154944e+06


c. Observability for CPU usage (with PromQL filtering)
```
sum(rate(container_cpu_usage_seconds_total[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load CPU usage time series csv files (from metrics explorer) for zonal and regional tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_zonal_cpu = get_timeseries('zonalCPUusage.csv')
# df_zonal_cpu

df_regional_cpu = get_timeseries('regionalCPUusage.csv')
# df_regional_cpu

In [ ]:
## Getting CPU usage for zonal Test

# Apply the function to each row in grouped_df
df_zonal_duration['cpu_usage_zonal'] = df_zonal_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_zonal_cpu), axis=1
)
# Select the desired columns using a list
df_zonal_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_zonal,bytes_received_zonal,cpu_usage_zonal
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,9148469248,7.602844e+07,13.665377
1,50,1.742906e+18,1.742906e+18,3/25/2025 20:37:00,3/25/2025 20:38:21,11995721728,2.795578e+08,25.622409
2,100,1.742907e+18,1.742907e+18,3/25/2025 20:49:03,3/25/2025 20:50:26,14524289024,9.674019e+05,9.692048
3,200,1.742908e+18,1.742908e+18,3/25/2025 21:06:21,3/25/2025 21:07:55,20613545984,1.331341e+07,15.838439
4,400,1.742908e+18,1.742908e+18,3/25/2025 21:10:47,3/25/2025 21:12:45,30477279232,2.409727e+06,35.290383
5,600,1.742909e+18,1.742909e+18,3/25/2025 21:16:39,3/25/2025 21:18:59,42199904256,3.105526e+06,39.092267


In [ ]:
## Getting CPU usage for regional Test

# Apply the function to each row in grouped_df
df_regional_duration['cpu_usage_regional'] = df_regional_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_regional_cpu), axis=1
)
# Select the desired columns using a list
df_regional_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_regional,bytes_received_regional,cpu_usage_regional
0,10,1.742900e+18,1.742900e+18,3/25/2025 18:46:25,3/25/2025 18:47:37,10142011392,3.095369e+07,9.149260
1,50,1.742900e+18,1.742900e+18,3/25/2025 18:51:35,3/25/2025 18:52:55,11428057088,1.192209e+08,17.988904
2,100,1.742876e+18,1.742876e+18,3/25/2025 12:18:30,3/25/2025 12:19:54,26949328896,2.055274e+06,18.997858
3,200,1.742902e+18,1.742902e+18,3/25/2025 19:19:21,3/25/2025 19:20:52,20127506432,1.207632e+07,14.628966
4,400,1.742902e+18,1.742902e+18,3/25/2025 19:24:20,3/25/2025 19:26:22,32191254528,2.644369e+06,23.932123
5,600,1.742903e+18,1.742903e+18,3/25/2025 19:44:41,3/25/2025 19:47:01,43523231744,3.154944e+06,43.055079


d. Observability for Memory usage (with PromQL filtering)
```
sum(avg_over_time(kubernetes_io:container_memory_used_bytes{monitored_resource="k8s_container"}[${__interval}]))
```
Open metrics explorer and extract all data scraped by this PromQL to csv. Refer [here](https://drive.google.com/file/d/1P3t_etDRgOxGrtgxYQF3bTLfhBf9-qYT/view?usp=sharing) for the steps. Load this csv to pandas dataframe.

In [ ]:
# Load CPU usage time series csv files (from metrics explorer) for zonal and regional tests
# Get these tests min datetime value and max datetime value (from test duration) and key in it to metrics explorer
# together with (b) PromQL command

df_zonal_memory = get_timeseries('zonalUsageMemory.csv')
# df_zonal_memory

df_regional_memory = get_timeseries('regionalUsageMemory.csv')
# df_regional_memory

In [ ]:
## Getting memory usage for zonal Test

# Apply the function to each row in grouped_df
df_zonal_duration['memory_usage_zonal'] = df_zonal_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_zonal_memory), axis=1
)
# Select the desired columns using a list
df_zonal_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_zonal,bytes_received_zonal,cpu_usage_zonal,memory_usage_zonal
0,10,1.742905e+18,1.742906e+18,3/25/2025 20:24:34,3/25/2025 20:25:46,9148469248,7.602844e+07,13.665377,9148469248
1,50,1.742906e+18,1.742906e+18,3/25/2025 20:37:00,3/25/2025 20:38:21,11995721728,2.795578e+08,25.622409,11995721728
2,100,1.742907e+18,1.742907e+18,3/25/2025 20:49:03,3/25/2025 20:50:26,14524289024,9.674019e+05,9.692048,14524289024
3,200,1.742908e+18,1.742908e+18,3/25/2025 21:06:21,3/25/2025 21:07:55,20613545984,1.331341e+07,15.838439,20613545984
4,400,1.742908e+18,1.742908e+18,3/25/2025 21:10:47,3/25/2025 21:12:45,30477279232,2.409727e+06,35.290383,30477279232
5,600,1.742909e+18,1.742909e+18,3/25/2025 21:16:39,3/25/2025 21:18:59,42199904256,3.105526e+06,39.092267,42199904256


In [ ]:
## Getting memory usage for regional Test

# Apply the function to each row in grouped_df
df_regional_duration['memory_usage_regional'] = df_regional_duration.apply(
    lambda row: find_max_telemetry(row['min_datetime_myt'], row['max_datetime_myt'], df_regional_memory), axis=1
)
# Select the desired columns using a list
df_regional_duration

,num_nodes,min,max,min_datetime_myt,max_datetime_myt,bytes_transmit_regional,bytes_received_regional,cpu_usage_regional,memory_usage_regional
0,10,1.742900e+18,1.742900e+18,3/25/2025 18:46:25,3/25/2025 18:47:37,10142011392,3.095369e+07,9.149260,10142011392
1,50,1.742900e+18,1.742900e+18,3/25/2025 18:51:35,3/25/2025 18:52:55,11428057088,1.192209e+08,17.988904,11428057088
2,100,1.742876e+18,1.742876e+18,3/25/2025 12:18:30,3/25/2025 12:19:54,26949328896,2.055274e+06,18.997858,26949328896
3,200,1.742902e+18,1.742902e+18,3/25/2025 19:19:21,3/25/2025 19:20:52,20127506432,1.207632e+07,14.628966,20127506432
4,400,1.742902e+18,1.742902e+18,3/25/2025 19:24:20,3/25/2025 19:26:22,32191254528,2.644369e+06,23.932123,32191254528
5,600,1.742903e+18,1.742903e+18,3/25/2025 19:44:41,3/25/2025 19:47:01,43523231744,3.154944e+06,43.055079,43523231744


In [ ]:
# Combine all results to get a table
df_zonal = df_zonal_duration[['num_nodes', 'bytes_transmit_zonal', 'bytes_received_zonal', 'cpu_usage_zonal', 'memory_usage_zonal']]
# df_zonal

df_regional = df_regional_duration[['num_nodes', 'bytes_transmit_regional', 'bytes_received_regional', 'cpu_usage_regional', 'memory_usage_regional']]
# df_regional

# Merge DataFrame
df_all = pd.merge(df_zonal,df_regional, on='num_nodes', how='outer')
df_all

,num_nodes,bytes_transmit_zonal,bytes_received_zonal,cpu_usage_zonal,memory_usage_zonal,bytes_transmit_regional,bytes_received_regional,cpu_usage_regional,memory_usage_regional
0,10,9148469248,7.602844e+07,13.665377,9148469248,10142011392,3.095369e+07,9.149260,10142011392
1,50,11995721728,2.795578e+08,25.622409,11995721728,11428057088,1.192209e+08,17.988904,11428057088
2,100,14524289024,9.674019e+05,9.692048,14524289024,26949328896,2.055274e+06,18.997858,26949328896
3,200,20613545984,1.331341e+07,15.838439,20613545984,20127506432,1.207632e+07,14.628966,20127506432
4,400,30477279232,2.409727e+06,35.290383,30477279232,32191254528,2.644369e+06,23.932123,32191254528
5,600,42199904256,3.105526e+06,39.092267,42199904256,43523231744,3.154944e+06,43.055079,43523231744


In [ ]:
def convert_units(df):
    """
    Converts the numeric columns of a DataFrame to human-readable units.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with converted units.
    """

    def human_readable_bytes(bytes_val):
        if bytes_val is None:
          return "N/A"
        units = ['B', 'KB', 'MB', 'GB', 'TB']
        bytes_val = float(bytes_val)
        i = 0
        while bytes_val >= 1024:
            bytes_val /= 1024
            i += 1
        return f"{bytes_val:.2f} {units[i]}"

    def human_readable_memory(bytes_val):
        if bytes_val is None:
          return "N/A"
        units = ['B', 'KB', 'MB', 'GB', 'TB']
        bytes_val = float(bytes_val)
        i = 0
        while bytes_val >= 1024:
            bytes_val /= 1024
            i += 1
        return f"{bytes_val:.2f} {units[i]}"

    def human_readable_cpu(cpu_val):
        if cpu_val is None:
          return "N/A"
        return f"{cpu_val:.2f}%"

    # Apply conversions
    df['bytes_transmit_zonal'] = df['bytes_transmit_zonal'].apply(human_readable_bytes)
    df['bytes_received_zonal'] = df['bytes_received_zonal'].apply(human_readable_bytes)
    df['cpu_usage_zonal'] = df['cpu_usage_zonal'].apply(human_readable_cpu)
    df['memory_usage_zonal'] = df['memory_usage_zonal'].apply(human_readable_memory)
    df['bytes_transmit_regional'] = df['bytes_transmit_regional'].apply(human_readable_bytes)
    df['bytes_received_regional'] = df['bytes_received_regional'].apply(human_readable_bytes)
    df['cpu_usage_regional'] = df['cpu_usage_regional'].apply(human_readable_cpu)
    df['memory_usage_regional'] = df['memory_usage_regional'].apply(human_readable_memory)

    return df

In [ ]:
# Convert units
df_converted = convert_units(df_all) #Use copy to avoid modifying the original dataframe
df_converted

,num_nodes,bytes_transmit_zonal,bytes_received_zonal,cpu_usage_zonal,memory_usage_zonal,bytes_transmit_regional,bytes_received_regional,cpu_usage_regional,memory_usage_regional
0,10,8.52 GB,72.51 MB,13.67%,8.52 GB,9.45 GB,29.52 MB,9.15%,9.45 GB
1,50,11.17 GB,266.61 MB,25.62%,11.17 GB,10.64 GB,113.70 MB,17.99%,10.64 GB
2,100,13.53 GB,944.73 KB,9.69%,13.53 GB,25.10 GB,1.96 MB,19.00%,25.10 GB
3,200,19.20 GB,12.70 MB,15.84%,19.20 GB,18.75 GB,11.52 MB,14.63%,18.75 GB
4,400,28.38 GB,2.30 MB,35.29%,28.38 GB,29.98 GB,2.52 MB,23.93%,29.98 GB
5,600,39.30 GB,2.96 MB,39.09%,39.30 GB,40.53 GB,3.01 MB,43.06%,40.53 GB
